In [223]:
# Homework 6 Template
# G. Besla & R. Li

In [224]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G
import os

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
# Step 1: modify CenterOfMass so that COM_P now takes a parameter specifying 
# by how much to decrease RMAX instead of a factor of 2
from CenterOfMass2 import CenterOfMass

In [ ]:
def OrbitCOM(galaxy, start, end, n):
    """function that loops over all the desired snapshots to compute the COM pos and vel as a function of time.
    inputs:
        galaxy: string
            name of the galaxy of interest
        start: integer
            number of the first snapshot to be read
        end: integer
            number of the last snapshot to be read
        n: integer
            intervals to return the center of mass   
    outputs: none, a text file will appear in your folder
    """
    
    # compose the filename for output
    fileout = "Orbit_" + galaxy + ".txt"
    
    #  set tolerance and VolDec for calculating COM_P in CenterOfMass
    # for M33 that is stripped more, use different values for VolDec
    delta = 0.1
    if galaxy == "M33":
        volDec = 4
    if galaxy == "MW" or "M31":
        volDec = 2 # Sometimes this returns "nan" for COM_p

    # generate the snapshot id sequence 
    # it is always a good idea to also check if the input is eligible (not required)
    snap_ids = np.arange(start, end+1, n)
    
    # initialize the array for orbital info: t, x, y, z, vx, vy, vz of COM
    orbit = np.zeros((len(snap_ids), 7))
    

    # a for loop 
    for  i, snap_id in enumerate(snap_ids): # loop over files
        
        # compose the data filename (be careful about the folder)
        # Add a string of the filenumber to the value “000”
        ilbl = '000' + str(snap_id)
        # Remove all but the last 3 digits
        ilbl = ilbl[-3:]
        # Name the file
        filename = "%s_"%(galaxy) + ilbl + '.txt'
        # Initialize an instance of CenterOfMass class, using disk particles
        COM = CenterOfMass(os.path.join("VLowRes", galaxy, filename), 2)
        # Store the COM pos and vel. Remember that now COM_P required VolDec
        COM_p = COM.COM_P(delta, volDec)
        COM_v = COM.COM_V(COM_p[0], COM_p[1], COM_p[2])
    
        # store the time, pos, vel in ith element of the orbit array, without units (.value) 
        # Extracting value
        time = COM.time.value
        x = COM_p[0].value
        y = COM_p[1].value
        z = COM_p[2].value
        vx = COM_v[0].value
        vy = COM_v[1].value
        vz = COM_v[2].value
        # PLacing values into orbit array
        orbit[i][0] = time
        orbit[i][1] = x
        orbit[i][2] = y
        orbit[i][3] = z
        orbit[i][4] = vx
        orbit[i][5] = vy
        orbit[i][6] = vz 
        # print snap_id to see the progress
        print(snap_id)

    # write the data to a file
    # we do this because we don't want to have to repeat this process 
    # this code should only have to be called once per galaxy.
    np.savetxt(fileout, orbit, fmt = "%11.3f"*7, comments='#',
               header= "{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
test = OrbitCOM("M33", 0, 100, 5)

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
Orbit_M33.txt


In [ ]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first! 


In [227]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt



In [228]:
# function to compute the magnitude of the difference between two vectors 
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit  


In [229]:
# Determine the magnitude of the relative position and velocities 

# of MW and M31

# of M33 and M31


In [230]:
# Plot the Orbit of the galaxies 
#################################



In [231]:
# Plot the orbital velocities of the galaxies 
#################################
